# **Normalizzazione dei Nomi Aziendali**

## **Introduzione**

Nel processo di deduplicazione e record linkage, è essenziale normalizzare i nomi aziendali per evitare variazioni dovute a abbreviazioni, formati o errori di battitura. In questa fase, implementiamo una strategia per rendere i nomi uniformi e comparabili.

---

## **Step 1: Importazione delle Librerie e del DataFrame**

In [61]:
import re
import pandas as pd
import unidecode

AZIENDE_CSV = '../aziende.csv'
IMPIEGATI_CSV = '../impiegati.csv'
companies_df = pd.read_csv(AZIENDE_CSV)

/tmp/ipykernel_85681/3147525914.py:7: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  companies_df = pd.read_csv(AZIENDE_CSV)


---

## **Step 2: Definizione della Mappa di Sostituzione**
Utilizziamo un dizionario per standardizzare suffissi comuni nei nomi aziendali.
### **Questo va sicuramente migliorato magari passando tutta la prima colonna a chat**

In [62]:
REPLACEMENTS = {
    r"&": "",
    r"(?<=\s|,|\.)inc\.?\b": "",
    r"(?<=\s|,|\.)ltd\b": "",
    r"(?<=\s|,|\.)llc\b": "",
    r"(?<=\s|,|\.)plc\b": "",
    r"(?<=\s|,|\.)co\.?\b": "",  # Handles "co." and "co,"
    r"(?<=\s|,|\.)corp\.?\b": "",
    r"(?<=\s|,|\.)srl\b": "",
    r"(?<=\s|,|\.)spa\b": "",
    r"(?<=\s|,|\.)ag\b": "",
    r"(?<=\s|,|\.)sa\b": "",
    r"(?<=\s|,|\.)ab\b": "",
    r"(?<=\s|,|\.)ou\b": "",
    r"(?<=\s|,|\.)as\b": "",
    r"(?<=\s|,|\.)gmbh\b": "",
    r"(?<=\s|,|\.)holding\b": "",
    r"(?<=\s|,|\.)holdings\b": "",
    r"(?<=\s|,|\.)corporation\b": "",
    r"(?<=\s|,|\.)industries\b": "",
    r"(?<=\s|,|\.)company limited\b": "",
    r"(?<=\s|,|\.)technology\b": "",
    r"(?<=\s|,|\.)solutions\b": "",
    r"(?<=\s|,|\.)systems\b": "",
    r"(?<=\s|,|\.)services\b": "",
    r"(?<=\s|,|\.)public company\b": "",

    # New patterns to remove dotted abbreviations
    r"\bs\.?p\.?a\.?\b": "",  # Handles "S.p.A." and variations
    r"\bs\.?r\.?l\.?\b": "",  # Handles "S.R.L."
    r"\bs\.?a\.?\b": "",      # Handles "S.A."
}

---

## **Step 3: Funzione di Normalizzazione**

Questa funzione applica una serie di trasformazioni per ottenere nomi uniformi:

- Conversione in minuscolo
- Rimozione di accenti
- Eliminazione di caratteri speciali
- Sostituzione dei suffissi tramite `REPLACEMENTS`

In [63]:
def normalize_name(name):
    """Normalizza i nomi delle aziende."""
    if pd.isna(name) or not isinstance(name, str):
        return ""
    
    # 1. Rimuove accenti e caratteri speciali
    name = unidecode.unidecode(name)
    
    # 2. Minuscolo e rimozione spazi extra
    name = name.lower().strip()
    
    # 3. Rimuove testo tra virgolette
    name = re.sub(r'"([^"]+)"','', name) # 3. Rimuove testo tra virgolette
    
    # 4. Applica le sostituzioni dalla mappa REPLACEMENTS
    for pattern, replacement in REPLACEMENTS.items():
        name = re.sub(pattern, replacement, name)
    
    # 5. Pulisce caratteri inutili
    name = re.sub(r"\s*,+\s*", " ", name)
    name = re.sub(r"\s*\.+\s*", " ", name) 
    
    # 6. Sostituisce spazi multipli con uno solo
    name = re.sub(r'\s+', ' ', name)  
    
    # 7. Eliminare punteggiatura indesiderata alla fine 
    name = re.sub(r"[.,\-]+$", "", name)
    
    return name.strip()

---

## **Step 4: Applicazione alla Colonna 'company_name'**

Eseguiamo la normalizzazione sulla colonna contenente i nomi aziendali.

In [64]:
companies_df['normalized_name'] = companies_df['company_name'].apply(normalize_name)
companies_df['modified'] = companies_df['normalized_name'] != companies_df['company_name']

---

## **Step 5: Visualizzazione e salvataggio del Risultato**

Mostriamo i primi risultati per verificare la trasformazione
e salva tutto in un csv

In [65]:
num_modified = companies_df['modified'].sum()
total_entries = len(companies_df)
percentage_modified = (num_modified / total_entries) * 100

print(f"Entries modificate: {num_modified} su {total_entries} ({percentage_modified:.2f}%)")

modified_entries = companies_df[companies_df['modified']][['company_name', 'normalized_name']]
display(modified_entries.sample(25))

companies_df = companies_df.sort_values(by=['normalized_name'])
companies_df[['company_name', 'normalized_name', 'modified']].to_csv('normalized_names.csv', index=False)

Entries modificate: 37656 su 76808 (49.03%)


,company_name,normalized_name
45809,noah holdings,noah
705,"acams north america chapters, inc.",acams north america chapters
47612,oriental pearl group co ltd,oriental pearl group
42969,monolithic power systems,monolithic power
41176,mec.line,mec line
2043,air transport services group,air transport group
35224,johnson service group plc,johnson service group
52384,pt. indah prakasa sentosa tbk,pt indah prakasa sentosa tbk
25852,ge t & d india,ge t d india
29241,hangzhou silan microelectronics co ltd,hangzhou silan microelectronics


---

## **Conclusioni**

Questo approccio garantisce una normalizzazione efficace dei nomi aziendali, riducendo le variazioni dovute a differenze di scrittura, abbreviazioni e formati. È facilmente estendibile aggiornando la mappa `REPLACEMENTS` con nuove regole di sostituzione.



In [66]:
companies_df['company_name'] = companies_df['normalized_name']
companies_df = companies_df.sort_values(by=['company_name'])
companies_df.to_csv('../aziende_normalizzate.csv', index=False)

# **Normalizzazione delle città**

In [67]:
filtered_df = companies_df[companies_df['city'].notna() & (companies_df['city'].str.strip() != "")].copy()
display(filtered_df[['company_name', 'city']].sample(25))

,company_name,city
58204,severn glocon (aberdeen),aberdeen
43676,mynt,taguig city
54779,renishaw p l c,wotton-under-edge
12272,cardekho,jaipur
16185,coocaa,shenzhen
56439,safety-kleen u k limited,brentford
6868,azienda ospedaliera santa croce e carle,cuneo
20109,eco logic brands,manteca
67531,total marketing support limited,london
68586,twofour broadcast limited,plymouth


### **Step 1: Eliminare eventuale testo tra parentesi** 
La cosa che sempra più intuitiva da fare è eliminare eventuale testo tra parentesi dopo la città (perdendo un po' di informazione, ma comunque accettabile)

In [68]:
def clean_city(city):
    if pd.isna(city) or not isinstance(city, str):
        return ""
    
    city = re.sub(r"\s*\(.*?\)\s*", "", city).strip()
    return city 

In [69]:
filtered_df['city'] = filtered_df['city'].apply(clean_city)
display(filtered_df[['company_name', 'city']].sample(25))

,company_name,city
27779,green investment group management limited,edinburgh
23342,finance birmingham limited,birmingham
54974,revolut,london
17016,cso oil gas (west africa) limited,jersey
14653,cisco international limited,feltham
65758,tenable network security netherlands b v,roma
28246,gs,milano
60543,societa' cattolica di assicurazione,verona
49278,paystand,scotts valley
41612,merci maman limited,london


In [70]:
# Apply the changes to the df
companies_df['city'] = companies_df['city'].apply(clean_city)

### **Step 2: Tradurre nomi di città italiane in italiano** 

Bisognerebbe anche indagare se qualche città viene riportata in lingue diverse...

In [71]:
# Query in italiano
milano_df = filtered_df[filtered_df["city"] == "milano"]
roma_df = filtered_df[filtered_df["city"] == "roma"]

# Query in inglese
milan_df = filtered_df[filtered_df["city"] == "milan"]
rome_df = filtered_df[filtered_df["city"] == "rome"]

In [72]:
display(milano_df[['company_name', 'city']])
display(milan_df[['company_name', 'city']])

,company_name,city
562,aberdeen standard investments ireland limited,milano
622,above comparison,milano
623,above comparison,milano
766,accenture,milano
765,accenture,milano
...,...,...
73454,withsecure,milano
74336,yes ticket,milano
74335,yes ticket,milano
75694,zurich italy bank,milano


,company_name,city
57140,satispay,milan
57364,scalapay,milan


In [73]:
display(roma_df[['company_name', 'city']])
display(rome_df[['company_name', 'city']])

,company_name,city
1431,aec underwriting agenzia di assicurazione e ri...,roma
1430,aec underwriting agenzia di assicurazione e ri...,roma
1739,agenzia delle entrate,roma
2463,ald automotive italia,roma
4282,angelini pharma italia aziende chimiche riunit...,roma
...,...,...
68023,trenitalia,roma
69746,universita' degli studi di roma la sapienza,roma
69748,universita' degli studi roma tre,roma
73045,western union payment ireland limited,roma


,company_name,city


Probabilmente, visto la sorgente dati, le aziende italiane sono state estratte da repository italiani o almeno convertite con città in italiano. Solo due aziende (*satispay* e *scalapay*) hanno il nome riportato in inglese.

Le città italiane sono segnate maggiormente in italiano possiamo pensare di **convertire le città con nome inglese in italiano** 

Abbiamo bisogno di un dizionario che mappi i nomi delle città (City-mapping). Evitiamo di scriverlo da zero e ne usiamo uno noto da qualche libreria (*do not reinvent the wheel...*).

Usiamo dunque la libreria `geopy` che con **GeoNames**, un database globale di località, ci permette di convertire i nomi 

In [86]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="city_normalizer")
city_cache = {}

In [90]:
def get_city_in_italian(city):
    if pd.isna(city) or not isinstance(city, str) or city == "":
        return ""

    if city in city_cache:
        print(f"Cache hit! {city}")
        return city_cache[city]
    
    try:
        location = geolocator.geocode(city, exactly_one=True, timeout=10)
        if location and "Italia" in location.address:
            location_it = geolocator.geocode(city, language="it", timeout=10)
            if location_it:
                print(f"translating {location_it}...")
                it_city = location_it.raw.get("display_name", city).split(",")[0].lower().strip()
                city_cache[city] = it_city
                return it_city
    except Exception as e:
        print(f"Errore con {city}: {e}")

    city_cache[city] = city
    return city

In [91]:
def apply_geocoding(city):
    result = get_city_in_italian(city)
    time.sleep(1)
    return result

In [ ]:
milan_df['city'] = milan_df['city'].apply(apply_geocoding)
display(milan_df[['company_name', 'city']])

**GeoNames** funziona, adesso applichiamolo all'intero dataframe

In [92]:
companies_df['city_translate'] = companies_df.apply(
    lambda row: apply_geocoding(row['city']), 
    axis=1
)
companies_df['modified'] = companies_df['city'] != companies_df['city_translate']

KeyboardInterrupt: 

In [58]:
num_modified = companies_df['modified'].sum()
total_cities = len(companies_df)
percentage_modified = (num_modified / total_cities) * 100

print(f"Città tradotte: {num_modified} su {total_cities} ({percentage_modified:.2f}%)")

translated_cities = companies_df[companies_df['modified']][['company_name', 'city', 'city_translate']]
display(translated_cities.sample(25))

companies_df = companies_df.sort_values(by=['company_name'])
companies_df[['company_name', 'city', 'city_translate']].to_csv('translated_cities.csv', index=False)

Città tradotte: 37656 su 76808 (49.03%)


KeyError: "['city_translate'] not in index"